# Pose Estimation Test

In [2]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

In [4]:
import tensorflow as tf
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 1480457730920788749
xla_global_id: -1
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 4163895296
locality {
  bus_id: 1
  links {
  }
}
incarnation: 14742043326740178508
physical_device_desc: "device: 0, name: NVIDIA GeForce GTX 1660 Ti, pci bus id: 0000:01:00.0, compute capability: 7.5"
xla_global_id: 416903419
]


In [5]:
import os

os.environ["CUDA_VISIBLE_DEVICES"] = '0' # 여러개 사용시 '0,1,2' 식으로 하나의 문자열에 입력
gpus = tf.config.experimental.list_physical_devices('GPU') # 호스트 러나임에 표시되는 GPU 장치 목록 반환

if gpus: # 반환된 GPU 장치 목록이 있다면
    try: # 해당 장치에 대한 메모리 증가 활성화 여부 설정
        tf.config.experimental.set_memory_growth(gpus[0], True)
    except RuntimeError as e: # try문 실패시에 에러문구 출력
        print(e)

In [6]:
print(tf.config.list_physical_devices('GPU'))

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [7]:
import tensorflow as tf
import tensorflow_hub as hub
import cv2
from matplotlib import pyplot as plt
from matplotlib import animation
import numpy as np
import pandas as pd
import math
import socket
import time

In [10]:
# import os
# os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [9]:
# ### Optional if you are using a GPU
# gpus = tf.config.experimental.list_physical_devices('GPU')
# for gpu in gpus:
#     tf.config.experimental.set_memory_growth(gpu, True)

In [16]:
# ### Optional if you are using a GPU
# gpus = tf.config.experimental.list_physical_devices('GPU')
# for gpu in gpus:
#     tf.config.experimental.set_memory_growth(gpu, True)

### Load Model
model = hub.load('https://tfhub.dev/google/movenet/multipose/lightning/1')
movenet = model.signatures['serving_default']

# ### Load Video
# video_path = '../../data/videos/'
# sampleVideo = video_path + 'sampleVideo.mp4'
# basicFour = video_path + 'basic_four.mp4'
# basicTwo = video_path + 'basic_two.mp4'
# swingDiff = video_path + 'swing_diff.mp4'
# swingSame = video_path + 'swing_same.mp4'
# swingRaw = video_path + 'swingRaw.mp4'
# temporalDifficult = video_path + 'temporalDifficultRaw.mp4'
# temporalEasy = video_path + 'temporalEasyRaw.mp4'



### Draw EDGES
EDGES = {
    (0, 1): 'm',
    (0, 2): 'c',
    (1, 3): 'm',
    (2, 4): 'c',
    (0, 5): 'm',
    (0, 6): 'c',
    (5, 7): 'm',
    (7, 9): 'm',
    (6, 8): 'c',
    (8, 10): 'c',
    (5, 6): 'y',
    (5, 11): 'm',
    (6, 12): 'c',
    (11, 12): 'y',
    (11, 13): 'm',
    (13, 15): 'm',
    (12, 14): 'c',
    (14, 16): 'c'
}

### Vector List
vectorList = [
    [0,1],
    [0,2],
    [1,3],
    [2,4],
    [3,5],
    [0,6],
    [1,7],
    [6,7],
    [6,8],
    [7,9],
    [8,10],
    [9,10]
]

# Function to loop through each person detected and render
def loop_through_people(frame, keypoints_with_scores, edges, confidence_threshold):
    for person in keypoints_with_scores:
        draw_connections(frame, person, edges, confidence_threshold)
        draw_keypoints(frame, person, confidence_threshold)


def draw_keypoints(frame, keypoints, confidence_threshold):
    y, x, c = frame.shape
    shaped = np.squeeze(np.multiply(keypoints, [y,x,1]))
    
    for kp in shaped:
        ky, kx, kp_conf = kp
        if kp_conf > confidence_threshold:
            cv2.circle(frame, (int(kx), int(ky)), 3, (0,255,0), -1)

def draw_connections(frame, keypoints, edges, confidence_threshold):
    y, x, c = frame.shape
    shaped = np.squeeze(np.multiply(keypoints, [y,x,1]))
    
    for edge, color in edges.items():
        p1, p2 = edge
        y1, x1, c1 = shaped[p1]
        y2, x2, c2 = shaped[p2]
        
        if (c1 > confidence_threshold) & (c2 > confidence_threshold):      
            cv2.line(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0,0,255), 2)


###
### Variables for Calculating FPS
prevTime = time.time() # previous time

if __name__ == "__main__":
    ### Variables
    numberOfPeople = 6
    lamdaVal = 1.0

    minBPD = 10.0
    maxBPD = 0.0
    
    saveNum = 0
    saveBPD = []

    ### Loading Video File
    cap = cv2.VideoCapture(0)    ### Change the File Here!!
    cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1280)
    cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 720)
    
    while cap.isOpened():
        ret, frame = cap.read()

        ### Variables for each frame
        BPD = []
        
        # Resize image
        img = frame.copy()
#         img = tf.image.resize_with_pad(tf.expand_dims(img, axis=0), 384,640)
        img = tf.image.resize_with_pad(tf.expand_dims(img, axis=0), 128, 256)

        input_img = tf.cast(img, dtype=tf.int32)
        
        # Detection section
        results = movenet(input_img)
        keypoints_with_scores = results['output_0'].numpy()[:,:,:51].reshape((6,17,3))
        keypoints_with_scores = keypoints_with_scores[:numberOfPeople]
        keypoints_only = np.delete(keypoints_with_scores,2,2)
        keypoints_only_body = np.delete(keypoints_only, [0,1,2,3,4], 1)
        
#         print(keypoints_with_scores)



        # Render keypoints 
        loop_through_people(frame, keypoints_with_scores, EDGES, 0.1)
#         loop_through_people(frame, [keypoints_with_scores[0]], EDGES, 0.1)    # Check for first person.....
        
        ### Calculate & Print FPS
        # Count Frame
        curTime = time.time()	# current time
        fps = 1 / (curTime - prevTime)
        prevTime = curTime
        # Save FPS
        fps_str = "FPS : %0.1f" %fps
        # FPS print
        font = cv2.FONT_HERSHEY_SIMPLEX
        location = (50, 200)
        cv2.putText(frame, fps_str, location, font, 1, (0, 255, 0), 2)
        
        cv2.imshow('Movenet Multipose', frame)
        
        
        if cv2.waitKey(10) & 0xFF==ord('q'):
            break
    
    cap.release()
    cv2.destroyAllWindows()

In [9]:
frame.shape

(960, 1280, 3)

# Load Tensorflow Model (fastest way)

In [21]:
# ### Optional if you are using a GPU
# gpus = tf.config.experimental.list_physical_devices('GPU')
# for gpu in gpus:
#     tf.config.experimental.set_memory_growth(gpu, True)

### Load Model
# model = hub.load('https://tfhub.dev/google/movenet/multipose/lightning/1')
model = tf.saved_model.load('../../model/moveNet/')
movenet = model.signatures['serving_default']

# ### Load Video
# video_path = '../../data/videos/'
# sampleVideo = video_path + 'sampleVideo.mp4'
# basicFour = video_path + 'basic_four.mp4'
# basicTwo = video_path + 'basic_two.mp4'
# swingDiff = video_path + 'swing_diff.mp4'
# swingSame = video_path + 'swing_same.mp4'
# swingRaw = video_path + 'swingRaw.mp4'
# temporalDifficult = video_path + 'temporalDifficultRaw.mp4'
# temporalEasy = video_path + 'temporalEasyRaw.mp4'



### Draw EDGES
EDGES = {
    (0, 1): 'm',
    (0, 2): 'c',
    (1, 3): 'm',
    (2, 4): 'c',
    (0, 5): 'm',
    (0, 6): 'c',
    (5, 7): 'm',
    (7, 9): 'm',
    (6, 8): 'c',
    (8, 10): 'c',
    (5, 6): 'y',
    (5, 11): 'm',
    (6, 12): 'c',
    (11, 12): 'y',
    (11, 13): 'm',
    (13, 15): 'm',
    (12, 14): 'c',
    (14, 16): 'c'
}

### Vector List
vectorList = [
    [0,1],
    [0,2],
    [1,3],
    [2,4],
    [3,5],
    [0,6],
    [1,7],
    [6,7],
    [6,8],
    [7,9],
    [8,10],
    [9,10]
]

# Function to loop through each person detected and render
def loop_through_people(frame, keypoints_with_scores, edges, confidence_threshold):
    for person in keypoints_with_scores:
        draw_connections(frame, person, edges, confidence_threshold)
        draw_keypoints(frame, person, confidence_threshold)


def draw_keypoints(frame, keypoints, confidence_threshold):
    y, x, c = frame.shape
    shaped = np.squeeze(np.multiply(keypoints, [y,x,1]))
    
    for kp in shaped:
        ky, kx, kp_conf = kp
        if kp_conf > confidence_threshold:
            cv2.circle(frame, (int(kx), int(ky)), 3, (0,255,0), -1)

def draw_connections(frame, keypoints, edges, confidence_threshold):
    y, x, c = frame.shape
    shaped = np.squeeze(np.multiply(keypoints, [y,x,1]))
    
    for edge, color in edges.items():
        p1, p2 = edge
        y1, x1, c1 = shaped[p1]
        y2, x2, c2 = shaped[p2]
        
        if (c1 > confidence_threshold) & (c2 > confidence_threshold):      
            cv2.line(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0,0,255), 2)


###
### Variables for Calculating FPS
prevTime = time.time() # previous time

if __name__ == "__main__":
    ### Variables
    numberOfPeople = 6
    lamdaVal = 1.0

    minBPD = 10.0
    maxBPD = 0.0
    
    saveNum = 0
    saveBPD = []

    ### Loading Video File
    cap = cv2.VideoCapture(0)    ### Change the File Here!!
    cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1280)
    cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 1280)

    
    while cap.isOpened():
        ret, frame = cap.read()

        ### Variables for each frame
        BPD = []
        
        # Resize image
        img = frame.copy()
#         img = tf.image.resize_with_pad(tf.expand_dims(img, axis=0), 384,640)
        img = tf.image.resize_with_pad(tf.expand_dims(img, axis=0), 256, 256)

        input_img = tf.cast(img, dtype=tf.int32)
        
        # Detection section
        results = movenet(input_img)
        keypoints_with_scores = results['output_0'].numpy()[:,:,:51].reshape((6,17,3))
        keypoints_with_scores = keypoints_with_scores[:numberOfPeople]
        keypoints_only = np.delete(keypoints_with_scores,2,2)
        keypoints_only_body = np.delete(keypoints_only, [0,1,2,3,4], 1)
        
#         print(keypoints_with_scores)



        # Render keypoints 
        loop_through_people(frame, keypoints_with_scores, EDGES, 0.1)
#         loop_through_people(frame, [keypoints_with_scores[0]], EDGES, 0.1)    # Check for first person.....
        
        ### Calculate & Print FPS
        # Count Frame
        curTime = time.time()	# current time
        fps = 1 / (curTime - prevTime)
        prevTime = curTime
        # Save FPS
        fps_str = "FPS : %0.1f" %fps
        # FPS print
        location = (50, 200)
        cv2.putText(frame, fps_str, (50,200), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
        
        cv2.imshow('Movenet Multipose', frame)
        
        
        if cv2.waitKey(10) & 0xFF==ord('q'):
            break
    
    cap.release()
    cv2.destroyAllWindows()

# Debugging

In [13]:
# ### Optional if you are using a GPU
# gpus = tf.config.experimental.list_physical_devices('GPU')
# for gpu in gpus:
#     tf.config.experimental.set_memory_growth(gpu, True)

### Load Model
# model = hub.load('https://tfhub.dev/google/movenet/multipose/lightning/1')
model = tf.saved_model.load('../../model/moveNet/')
movenet = model.signatures['serving_default']

# ### Load Video
# video_path = '../../data/videos/'
# sampleVideo = video_path + 'sampleVideo.mp4'
# basicFour = video_path + 'basic_four.mp4'
# basicTwo = video_path + 'basic_two.mp4'
# swingDiff = video_path + 'swing_diff.mp4'
# swingSame = video_path + 'swing_same.mp4'
# swingRaw = video_path + 'swingRaw.mp4'
# temporalDifficult = video_path + 'temporalDifficultRaw.mp4'
# temporalEasy = video_path + 'temporalEasyRaw.mp4'



### Draw EDGES
EDGES = {
    (0, 1): 'm',
    (0, 2): 'c',
    (1, 3): 'm',
    (2, 4): 'c',
    (0, 5): 'm',
    (0, 6): 'c',
    (5, 7): 'm',
    (7, 9): 'm',
    (6, 8): 'c',
    (8, 10): 'c',
    (5, 6): 'y',
    (5, 11): 'm',
    (6, 12): 'c',
    (11, 12): 'y',
    (11, 13): 'm',
    (13, 15): 'm',
    (12, 14): 'c',
    (14, 16): 'c'
}

### Vector List
vectorList = [
    [0,1],
    [0,2],
    [1,3],
    [2,4],
    [3,5],
    [0,6],
    [1,7],
    [6,7],
    [6,8],
    [7,9],
    [8,10],
    [9,10]
]

# Function to loop through each person detected and render
def loop_through_people(frame, keypoints_with_scores, edges, confidence_threshold):
    for person in keypoints_with_scores:
        draw_connections(frame, person, edges, confidence_threshold)
        draw_keypoints(frame, person, confidence_threshold)


def draw_keypoints(frame, keypoints, confidence_threshold):
    y, x, c = frame.shape
    shaped = np.squeeze(np.multiply(keypoints, [y,x,1]))
    
    for kp in shaped:
        ky, kx, kp_conf = kp
        if kp_conf > confidence_threshold:
            cv2.circle(frame, (int(kx), int(ky)), 3, (0,255,0), -1)

def draw_connections(frame, keypoints, edges, confidence_threshold):
    y, x, c = frame.shape
    shaped = np.squeeze(np.multiply(keypoints, [y,x,1]))
    
    for edge, color in edges.items():
        p1, p2 = edge
        y1, x1, c1 = shaped[p1]
        y2, x2, c2 = shaped[p2]
        
        if (c1 > confidence_threshold) & (c2 > confidence_threshold):      
            cv2.line(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0,0,255), 2)


###
### Variables for Calculating FPS
prevTime = time.time() # previous time

if __name__ == "__main__":
    ### Variables
    numberOfPeople = 6
    lamdaVal = 1.0

    minBPD = 10.0
    maxBPD = 0.0
    
    saveNum = 0
    saveBPD = []

    ### Loading Video File
    cap = cv2.VideoCapture(0)    ### Change the File Here!!
    cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
    cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 320)
    

    
    while cap.isOpened():
        ret, frame = cap.read()

        ### Variables for each frame
        initialTime = time.time()
        
        # Resize image
        img = frame.copy()
#         img = tf.image.resize_with_pad(tf.expand_dims(img, axis=0), 384,640)
        img = tf.image.resize_with_pad(tf.expand_dims(img, axis=0), 128, 256)

        input_img = tf.cast(img, dtype=tf.int32)
        
        frame = cv2.resize(frame, (1280, 640))

        
        # Debug
        firstTime = time.time()-initialTime
        cv2.putText(frame, f'1: {round(firstTime,3)}', (50,100), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255,0), 2)

        
        # Detection section
        results = movenet(input_img)
        
        # Debug
        secondTime = time.time()-initialTime
        cv2.putText(frame, f'2: {round(secondTime,3)}', (50,150), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255,0), 2)

        
        keypoints_with_scores = results['output_0'].numpy()[:,:,:51].reshape((6,17,3))
        keypoints_with_scores = keypoints_with_scores[:numberOfPeople]
        keypoints_only = np.delete(keypoints_with_scores,2,2)
        keypoints_only_body = np.delete(keypoints_only, [0,1,2,3,4], 1)
        
#         print(keypoints_with_scores)
    

        # Render keypoints 
        loop_through_people(frame, keypoints_with_scores, EDGES, 0.1)
#         loop_through_people(frame, [keypoints_with_scores[0]], EDGES, 0.1)    # Check for first person.....
        
        # Debug
        thirdTime = time.time()-initialTime
        cv2.putText(frame, f'3: {round(thirdTime,3)}', (50,200), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255,0), 2)

    
        ### Calculate & Print FPS
        # Count Frame
        curTime = time.time()	# current time
        fps = 1 / (curTime - prevTime)
        prevTime = curTime
        # Save FPS
        fps_str = "FPS : %0.1f" %fps
        # FPS print
        cv2.putText(frame, fps_str, (50,50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
        
        
        # Debug
        fourthTime = time.time()-initialTime
        cv2.putText(frame, f'3: {round(fourthTime,3)}', (50,250), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255,0), 2)
        
        cv2.imshow('Movenet Multipose', frame)
        
        
        if cv2.waitKey(10) & 0xFF==ord('q'):
            break
    
    cap.release()
    cv2.destroyAllWindows()

# Raw Webcam

In [11]:
# ### Optional if you are using a GPU
# gpus = tf.config.experimental.list_physical_devices('GPU')
# for gpu in gpus:
#     tf.config.experimental.set_memory_growth(gpu, True)

# ### Load Model
# # model = hub.load('https://tfhub.dev/google/movenet/multipose/lightning/1')
# model = tf.saved_model.load('../../model/moveNet/')
# movenet = model.signatures['serving_default']

# ### Load Video
# video_path = '../../data/videos/'
# sampleVideo = video_path + 'sampleVideo.mp4'
# basicFour = video_path + 'basic_four.mp4'
# basicTwo = video_path + 'basic_two.mp4'
# swingDiff = video_path + 'swing_diff.mp4'
# swingSame = video_path + 'swing_same.mp4'
# swingRaw = video_path + 'swingRaw.mp4'
# temporalDifficult = video_path + 'temporalDifficultRaw.mp4'
# temporalEasy = video_path + 'temporalEasyRaw.mp4'



### Draw EDGES
EDGES = {
    (0, 1): 'm',
    (0, 2): 'c',
    (1, 3): 'm',
    (2, 4): 'c',
    (0, 5): 'm',
    (0, 6): 'c',
    (5, 7): 'm',
    (7, 9): 'm',
    (6, 8): 'c',
    (8, 10): 'c',
    (5, 6): 'y',
    (5, 11): 'm',
    (6, 12): 'c',
    (11, 12): 'y',
    (11, 13): 'm',
    (13, 15): 'm',
    (12, 14): 'c',
    (14, 16): 'c'
}

### Vector List
vectorList = [
    [0,1],
    [0,2],
    [1,3],
    [2,4],
    [3,5],
    [0,6],
    [1,7],
    [6,7],
    [6,8],
    [7,9],
    [8,10],
    [9,10]
]

# Function to loop through each person detected and render
def loop_through_people(frame, keypoints_with_scores, edges, confidence_threshold):
    for person in keypoints_with_scores:
        draw_connections(frame, person, edges, confidence_threshold)
        draw_keypoints(frame, person, confidence_threshold)


def draw_keypoints(frame, keypoints, confidence_threshold):
    y, x, c = frame.shape
    shaped = np.squeeze(np.multiply(keypoints, [y,x,1]))
    
    for kp in shaped:
        ky, kx, kp_conf = kp
        if kp_conf > confidence_threshold:
            cv2.circle(frame, (int(kx), int(ky)), 3, (0,255,0), -1)

def draw_connections(frame, keypoints, edges, confidence_threshold):
    y, x, c = frame.shape
    shaped = np.squeeze(np.multiply(keypoints, [y,x,1]))
    
    for edge, color in edges.items():
        p1, p2 = edge
        y1, x1, c1 = shaped[p1]
        y2, x2, c2 = shaped[p2]
        
        if (c1 > confidence_threshold) & (c2 > confidence_threshold):      
            cv2.line(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0,0,255), 2)


###
### Variables for Calculating FPS
prevTime = time.time() # previous time

if __name__ == "__main__":
    ### Variables
#     numberOfPeople = 6
#     lamdaVal = 1.0

#     minBPD = 10.0
#     maxBPD = 0.0
    
#     saveNum = 0
#     saveBPD = []

    ### Loading Video File
    cap = cv2.VideoCapture(0)    ### Change the File Here!!
    cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
    cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 320)
#     cap.set(cv2.CAP_PROP_FOURCC, cv2.VideoWriter_fourcc(*"MJPG")) # add this line
    
    
#     fourcc = cv2.VideoWriter_fourcc(*'mp4v')
#     out = cv2.VideoWriter(output_file, fourcc, output_fps,
#                       (int(cap.get(3)), int(cap.get(4))))

    
    while cap.isOpened():
        ret, frame = cap.read()
        
        frame = cv2.resize(frame, (1280, 640))

        ### Variables for each frame
        initialTime = time.time()
        
#         # Resize image
#         img = frame.copy()
# #         img = tf.image.resize_with_pad(tf.expand_dims(img, axis=0), 384,640)
#         img = tf.image.resize_with_pad(tf.expand_dims(img, axis=0), 256, 256)

#         input_img = tf.cast(img, dtype=tf.int32)
        
        # Debug
        firstTime = time.time()-initialTime
        cv2.putText(frame, f'1: {round(firstTime,3)}', (50,100), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255,0), 2)

        
        # Detection section
#         results = movenet(input_img)
        
        # Debug
        secondTime = time.time()-initialTime
        cv2.putText(frame, f'2: {round(secondTime,3)}', (50,150), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255,0), 2)

        
#         keypoints_with_scores = results['output_0'].numpy()[:,:,:51].reshape((6,17,3))
#         keypoints_with_scores = keypoints_with_scores[:numberOfPeople]
#         keypoints_only = np.delete(keypoints_with_scores,2,2)
#         keypoints_only_body = np.delete(keypoints_only, [0,1,2,3,4], 1)
        
#         print(keypoints_with_scores)
    

        # Render keypoints 
#         loop_through_people(frame, keypoints_with_scores, EDGES, 0.1)
#         loop_through_people(frame, [keypoints_with_scores[0]], EDGES, 0.1)    # Check for first person.....
        
        # Debug
        thirdTime = time.time()-initialTime
        cv2.putText(frame, f'3: {round(thirdTime,3)}', (50,200), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255,0), 2)

    
        ### Calculate & Print FPS
        # Count Frame
        curTime = time.time()	# current time
        fps = 1 / (curTime - prevTime)
        prevTime = curTime
        # Save FPS
        fps_str = "FPS : %0.1f" %fps
        # FPS print
        cv2.putText(frame, fps_str, (50,50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
        
        
        # Debug
        fourthTime = time.time()-initialTime
        cv2.putText(frame, f'3: {round(fourthTime,3)}', (50,250), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255,0), 2)
        
        cv2.imshow('Movenet Multipose', frame)
        
        
        if cv2.waitKey(1) & 0xFF==ord('q'):
            break
    
    cap.release()
    cv2.destroyAllWindows()

In [41]:
# import cv2
# import time
# from imutils.video import VideoStream


# fvs = VideoStream(src=0).start()

# time.sleep(1.0)

# while True:
#     frame = fvs.read()

#     cv2.imshow("Frame", frame)

KeyboardInterrupt: 

In [ ]:
# import the necessary packages
from imutils.video import FileVideoStream
from imutils.video import FPS
import numpy as np
import argparse
import imutils
import time
import cv2
# construct the argument parse and parse the arguments
ap = argparse.ArgumentParser()
ap.add_argument("-v", "--video", required=True,help="path to input video file")
args = vars(ap.parse_args())
# start the file video stream thread and allow the buffer to
# start to fill
print("[INFO] starting video file thread...")
fvs = FileVideoStream(args["video"]).start()
time.sleep(1.0)
# start the FPS timer
fps = FPS().start()

# loop over frames from the video file stream
while fvs.more():
    # grab the frame from the threaded video file stream, resize
    # it, and convert it to grayscale (while still retaining 3
    # channels)
    frame = fvs.read()
    frame = imutils.resize(frame, width=450)
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    frame = np.dstack([frame, frame, frame])
    # display the size of the queue on the frame
    cv2.putText(frame, "Queue Size: {}".format(fvs.Q.qsize()),(10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)
    # show the frame and update the FPS counter
    cv2.imshow("Frame", frame)
    cv2.waitKey(1)
    fps.update()
    
# stop the timer and display FPS information
fps.stop()
print("[INFO] elasped time: {:.2f}".format(fps.elapsed()))
print("[INFO] approx. FPS: {:.2f}".format(fps.fps()))
# do a bit of cleanup
cv2.destroyAllWindows()
fvs.stop()

In [9]:
import cv2
import tkinter as tk
from PIL import Image, ImageTk

# Set the desired resolution
width, height = 640, 320

# Create a Tkinter window
root = tk.Tk()
root.title("Webcam Feed")

# Create a Label widget to display the webcam feed
label = tk.Label(root)
label.pack()

# Initialize the webcam
cap = cv2.VideoCapture(0)
cap.set(3, width)
cap.set(4, height)

def update_frame():
    ret, frame = cap.read()
    if ret:
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        frame = cv2.resize(frame, (width * 2, height * 2))  # Scale up the frame
        img = Image.fromarray(frame)
        img_tk = ImageTk.PhotoImage(image=img)
        label.img = img_tk
        label.config(image=img_tk)
    label.after(10, update_frame)

update_frame()  # Start updating the frame

root.mainloop()
cap.release()

Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\Users\LeeSooHwan\anaconda3\envs\cheer\lib\tkinter\__init__.py", line 1892, in __call__
    return self.func(*args)
  File "C:\Users\LeeSooHwan\anaconda3\envs\cheer\lib\tkinter\__init__.py", line 814, in callit
    func(*args)
  File "C:\Users\LeeSooHwan\AppData\Local\Temp\ipykernel_46512\1256664166.py", line 27, in update_frame
    img_tk = ImageTk.PhotoImage(image=img)
  File "C:\Users\LeeSooHwan\anaconda3\envs\cheer\lib\site-packages\PIL\ImageTk.py", line 128, in __init__
    self.paste(image)
  File "C:\Users\LeeSooHwan\anaconda3\envs\cheer\lib\site-packages\PIL\ImageTk.py", line 180, in paste
    image.convert2(block, image)  # convert directly between buffers
KeyboardInterrupt


KeyboardInterrupt: 